In [1]:
import rpy2.robjects as robjects
r = robjects.r

r.load('../data.txt')

<StrVector - Python:0x11041dbd8 / R:0x7ff1845bd680>
[str, str, str]

In [2]:
# print(r.U.dot(r.t(r.V)))

In [2]:
# print(r.V)

In [3]:
# from rpy2.robjects import pandas2ri
# pandas2ri.activate()

In [4]:
# pU = pandas2ri.ri2py(r.U)
# pV = pandas2ri.ri2py(r.V)

In [5]:
# pU.dot(pV.T)

In [6]:
import numpy as np

In [7]:
# DPP
def sample_k(items, L, k, max_nb_iterations=1000, rng=np.random):
    """
    Sample a list of k items from a DPP defined
    by the similarity matrix L. The algorithm
    is iterative and runs for max_nb_iterations.
    The algorithm used is from
    (Fast Determinantal Point Process Sampling withw
    Application to Clustering, Byungkon Kang, NIPS 2013)
    """
    initial = rng.choice(range(len(items)), size=k, replace=False)
    X = [False] * len(items)
    for i in initial:
        X[i] = True
    X = np.array(X)
    for i in range(max_nb_iterations):
        u = rng.choice(np.arange(len(items))[X])
        v = rng.choice(np.arange(len(items))[~X])
        Y = X.copy()
        Y[u] = False
        L_Y = L[Y, :]
        L_Y = L_Y[:, Y]
        L_Y_inv = np.linalg.inv(L_Y)

        c_v = L[v:v+1, :]
        c_v = c_v[:, v:v+1]
        b_v = L[Y, :]
        b_v = b_v[:, v:v+1]
        c_u = L[u:u+1, :]
        c_u = c_u[:, u:u+1]
        b_u = L[Y, :]
        b_u = b_u[:, u:u+1]

        p = min(1, c_v - np.dot(np.dot(b_v.T, L_Y_inv), b_v) /
                (c_u - np.dot(np.dot(b_u.T, L_Y_inv.T), b_u)))
        if rng.uniform() <= p:
            X = Y[:]
            X[v] = True
    return np.array(items)[X]

In [8]:
similarity = np.array(r.V.dot(r.V.transpose()))
chosen = sample_k(range(r.V.dim[0]), similarity, 5)

In [9]:
import os, sys
os.chdir('..')
from my_io import Dataset
dataset = Dataset('fraction')

prefix is now new-fraction0
prefix is now new-fraction1
prefix is now new-fraction2
prefix is now new-fraction3
prefix is now new-fraction4
prefix is now new-fraction0


In [10]:
nb_zero = 0
nb_full = 0
total = 0
dame_zero = 0
dame_full = 0
ok = 0
# chosen = random.sample(range(20), 5)
# chosen = sample_k(range(r.V.dim[0]), similarity, 5)
for line in np.array(dataset.data):
    if sum(line) == 0:
        nb_zero += 1
    elif sum(line) == 20:
        nb_full += 1
    else:
        # chosen = random.sample(range(20), 5)
        # chosen = sample_k(range(r.V.dim[0]), similarity, 5)
        total += 1
        if sum(line[chosen]) == 0:
            dame_zero += 1
        elif sum(line[chosen]) == 5:
            dame_full += 1
        else:
            ok += 1
print(nb_zero, 'ont tout faux', nb_full, 'ont tout juste, sinon', total)
print(dame_zero, dame_full, ok) # 156, 79, 258 pour chosen 1 2 10 12 16
# (63, 86, 344) pour full random
# (105, 51, 337) pour once random
# (59, 123, 311) pour dpp for everyone

(13, 'ont tout faux', 30, 'ont tout juste, sinon', 493)
(65, 148, 280)


In [11]:
from calc import avgstd

In [12]:
import random

In [13]:
for _ in range(10):
    print(sample_k(range(r.V.dim[0]), similarity, 5))

[ 3  5  7 10 19]
[ 0  3  5 13 15]
[ 4  5  7 10 11]


/Users/jin/code/lib/python2.7/site-packages/ipykernel/__main__.py:35: RuntimeWarning: divide by zero encountered in divide


[ 1  4  7 11 15]
[1 2 5 6 8]
[ 2  5  7  9 11]
[ 7  8  9 10 15]
[ 1  2  5  8 11]
[ 3  5  8 15 19]
[ 3 10 13 15 19]


In [59]:
user = np.array(random.choice(dataset.data))
user

array([False, False, False, False, False, False,  True,  True, False,
       False, False,  True, False,  True,  True, False, False,  True,
       False, False], dtype=bool)

In [63]:
chosen = sample_k(range(r.V.dim[0]), similarity, 5)
chosen

array([ 3,  5, 13, 16, 19])

In [16]:
user[chosen]

array([ True, False, False, False, False], dtype=bool)

In [17]:
raw_data = map(int, reduce(lambda x, y: x + y, dataset.data))
raw_data
nb_students = len(dataset.data)
data = r.matrix(robjects.IntVector(raw_data), nrow=nb_students, byrow=True)
robjects.globalenv['data'] = data

In [66]:
from rpy2.robjects.packages import importr
mirtCAT = importr('mirtCAT')

def bootstrap(chosen, answers):
    items_asked = ','.join(map(lambda x: str(x + 1), chosen))
    answers_got = ','.join(map(lambda x: str(int(x)), answers))
    r("CATdesign <- mirtCAT(NULL, fit, criteria='Drule', start_item='Drule', local_pattern=data, design_elements=TRUE)")
    r("CATdesign <- updateDesign(CATdesign, items=c(%s), response=c(%s))" % (items_asked, answers_got))
    r("CATdesign$person$Update.thetas(CATdesign$design, CATdesign$test)")
    r("theta <- cbind(CATdesign$person$thetas, 1)")
    return r.theta

chosen = range(20)
bootstrap(chosen, user[chosen])

<Matrix - Python:0x114133a70 / R:0x7ff189398f50>
[0.824530, -0.442841, -0.406743, ..., -0.312825, 0.006750, 1.000000]

In [67]:
print(r.theta)

            F1         F2         F3         F4         F5        F6         F7
[1,] 0.8245297 -0.4428409 -0.4067433 -0.9251847 -0.3203103 0.3622759 -0.3128252
              F8  
[1,] 0.006749974 1



In [68]:
Z = r.theta.dot(r.V.transpose())
from scipy.special import expit
pred = expit(Z).ravel()
print(pred[chosen])
print(chosen)
print(user[chosen])
for p, t in zip(pred, user):
    print('OK' if round(p) == t else 'NOK', p, t)

[ 0.30730042  0.28153148  0.23584139  0.19507744  0.21245401  0.69938945
  0.35321188  0.63827486  0.54701808  0.15537779  0.21497252  0.62431666
  0.04986442  0.50777575  0.51795291  0.48275067  0.18502337  0.26698703
  0.16002217  0.12283328]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[False False False False False False  True  True False False False  True
 False  True  True False False  True False False]
('OK', 0.30730042202082869, False)
('OK', 0.28153148402642558, False)
('OK', 0.23584138561231235, False)
('OK', 0.19507744370608079, False)
('OK', 0.21245401302329425, False)
('NOK', 0.69938945136706421, False)
('NOK', 0.35321187653670727, True)
('OK', 0.63827486120235677, True)
('NOK', 0.5470180770047478, False)
('OK', 0.15537778879559777, False)
('OK', 0.21497252019487537, False)
('OK', 0.62431666351109727, True)
('OK', 0.049864423474863646, False)
('OK', 0.50777574665157355, True)
('OK', 0.51795291361484885, True)
('OK', 0.48275067317660797, False)
('O

In [40]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(fit_intercept=True, solver='lbfgs')
X = np.array(r.V)[chosen, 0:8]
clf.fit(X, user[chosen], sample_weight=np.array(r.V)[chosen, 8])
pred = clf.coef_[0]
print(clf.coef_, clf.intercept_)
clf.predict_proba(np.array(r.V)[chosen, 0:8])

(array([[ 0.        ,  0.11454297,  0.13095647,  0.        ,  0.14525113,
         0.        ,  0.05413768,  0.        ]]), array([-2.4988934]))


array([[ 0.88756216,  0.11243784],
       [ 0.91744323,  0.08255677],
       [ 0.91609395,  0.08390605],
       [ 0.89780998,  0.10219002],
       [ 0.88718572,  0.11281428]])

In [32]:
user[chosen]

array([ True, False, False, False, False], dtype=bool)

In [49]:
nb_students = len(dataset.data)
nb_questions = len(dataset.data[0])
mean_success = [float(sum(dataset.data[i][j] for i in range(nb_students))) / nb_students for j in range(nb_questions)]
sorted([(abs(mean_success[i] - 0.5), i) for i in range(nb_questions)])[:5]

[(0.014925373134328401, 2),
 (0.026119402985074647, 10),
 (0.027985074626865725, 3),
 (0.03171641791044777, 0),
 (0.03171641791044777, 17)]

In [50]:
chosen = [0, 2, 3, 10, 17]